
## 实战项目：图像标注

在这个notebook中，训练CNN-RNN模型,搜索好的模型时尝试多种不同的架构和超参数。
通过点击以下链接导航到该notebook：
- [Step 1](#step1): 训练设置
- [Step 2](#step2): 训练你的模型

<a id='step1'></a>
## Step 1: 训练设置

在该notebook的此步骤中，需要通过定义超参数并设置训练过程中重要的其他选项来自定义对CNN-RNN模型的训练。
### 任务 #1

首先，请设置以下变量：
- `batch_size` - 每个训练批次的批次大小。它是指用于在每个训练步骤中修改模型权重的图像标注对的数量。
- `vocab_threshold` - 单词阈值最小值。请注意，阈值越大，词汇量越小，而阈值越小，则表示将包括较少的词汇，词汇量则越大。
- `vocab_from_file` - 一个布尔值，用于决定是否从文件加载词汇表。
- `embed_size` - the dimensionality of the image and word embeddings.  图像和单词嵌入的维度。
- `hidden_size` - RNN解码器隐藏状态下的特征数。
- `num_epochs` - 训练模型的epoch数。我们建议你设置为`num_epochs=3`，但可以根据需要随意增加或减少此数字。 [这篇论文](https://arxiv.org/pdf/1502.03044.pdf) 在一个最先进的GPU上对一个标注生成模型训练了3天，但很快你就会发现，其实在几个小时内就可以得到合理的结果！（_但是，如果你想让你的模型与当前的研究一较高下，则需要更长时间的训练。_)
- `save_every` - 确定保存模型权重的频率。我们建议你设置为`save_every=1`，便于在每个epoch后保存模型权重。这样，在第`i`个epoch之后，编码器和解码器权重将在`models/`文件夹中分别保存为`encoder-i.pkl`和`decoder-i.pkl`。
- `print_every` - 确定在训练时将批次损失输出到Jupyter notebook的频率。请注意，训练时，你**将不会**看到损失函数的单调减少，这一点非常好并且完全可以预料到！我们建议你将其保持在默认值`100` ，从而避免让这个notebook运行变慢，但之后随时都可以进行更改。
- `log_file` - 包含每个步骤中训练期间的损失与复杂度演变过程的的文本文件的名称。

对于上述某些值，可以仔细阅读 [这篇文章](https://arxiv.org/pdf/1502.03044.pdf) 与 [这篇文章](https://arxiv.org/pdf/1411.4555.pdf) ，获得有用的指导！

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

batch_size =128          # 批量
vocab_threshold = 5.        # 阈值
vocab_from_file = True  # 是否使用词汇字典文件
embed_size =512           # 词嵌入大小
hidden_size = 512          #隐藏层大小
num_epochs = 3             # epochs
save_every = 1             
print_every = 100        
log_file = 'training_log.txt'       # 保存训练过程的数据

transform_train = transforms.Compose([ 
    transforms.Resize(256), 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),              
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))])

data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

vocab_size = len(data_loader.dataset.vocab)
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.embed.parameters())+ list(encoder.bn.parameters())
optimizer=torch.optim.Adam(params)
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.81s)
creating index...


  0%|          | 1027/414113 [00:00<01:25, 4845.99it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:10<00:00, 5867.67it/s]
/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<a id='step2'></a>
## Step 2: 训练你的模型

在**Step 1**中执行代码单元格后，下面的训练过程应该就不会出现问题了。
使用加载已保存的权重来恢复训练很有用。使用下面的代码行加载权重：
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

### 关于调整超参数的说明

但是，这样无法知道模型是否过度拟合训练数据,过度拟合是训练图像标注模型时常会遇到的问题。可以阅读 [本文](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636)4.3.1节中最小化过度拟合的一些方法。

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    for i_step in range(1, total_step+1):
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        images, captions = next(iter(data_loader))
        images = images.to(device)
        captions = captions.to(device)
        decoder.zero_grad()
        encoder.zero_grad()
        features = encoder(images)
        outputs = decoder(features, captions)
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        loss.backward()
        optimizer.step()
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        print('\r' + stats, end="")
        sys.stdout.flush()
        f.write(stats + '\n')
        f.flush()
        if i_step % print_every == 0:
            print('\r' + stats)
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
f.close()

Epoch [1/3], Step [100/3236], Loss: 4.1022, Perplexity: 60.4745
Epoch [1/3], Step [200/3236], Loss: 3.3935, Perplexity: 29.7695
Epoch [1/3], Step [300/3236], Loss: 3.2420, Perplexity: 25.58538
Epoch [1/3], Step [400/3236], Loss: 2.9838, Perplexity: 19.7632
Epoch [1/3], Step [500/3236], Loss: 2.9309, Perplexity: 18.7439
Epoch [1/3], Step [600/3236], Loss: 3.1366, Perplexity: 23.0264
Epoch [1/3], Step [700/3236], Loss: 2.7119, Perplexity: 15.05797
Epoch [1/3], Step [800/3236], Loss: 2.5724, Perplexity: 13.0969
Epoch [1/3], Step [900/3236], Loss: 3.4026, Perplexity: 30.0429
Epoch [1/3], Step [1000/3236], Loss: 2.5100, Perplexity: 12.3044
Epoch [1/3], Step [1100/3236], Loss: 2.3984, Perplexity: 11.0054
Epoch [1/3], Step [1200/3236], Loss: 2.6150, Perplexity: 13.6675
Epoch [1/3], Step [1300/3236], Loss: 2.4940, Perplexity: 12.1098
Epoch [1/3], Step [1400/3236], Loss: 2.4189, Perplexity: 11.2337
Epoch [1/3], Step [1500/3236], Loss: 2.4296, Perplexity: 11.3548
Epoch [1/3], Step [1600/3236], L